In [1]:
import os
import time
import random
import math
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from PIL import Image
from IPython.display import clear_output

import ai2thor
import ai2thor_colab
from ai2thor_colab import plot_frames
from ai2thor.controller import Controller

from ai2thor.platform import CloudRendering
controller = Controller(platform=CloudRendering)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torch.distributions import Categorical

In [3]:
from network import DQN
from utils import (
    to_torchdim, frame2tensor, plot_durations, plot_durations_w_shaded,
    encode_feedback_find, encode_feedback_find_for_pickup,
    encode_feedback_pickup, encode_feedback_pickup_for_go,
    encode_feedback_go, encode_feedback_go_for_throw, 
    encode_feedback_throw
)

In [4]:
import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')

## Set Environment

In [5]:
floor_index = random.randint(0, 30)
floor_index = 20

controller = Controller(
    agentMode = "default", # arm
    visibilityDistance = 0.50,
    scene = f"FloorPlan{floor_index}",

    # step sizes
    snapToGrid = True,
    gridSize = 0.25,
    rotateStepDegrees = 90,

    # image modalities
    renderInstanceSegmentation = False,
    renderDepthImage = False,
    renderSemanticSegmentation = False,
    renderNormalsImage = False,
    
    # camera properties
    width = 600,
    height = 420,
    fieldOfView = 120,
    
    # set seed for reproducability
    seed = 90,
)

plot_frames(controller.last_event)

In [6]:
instructions = [("find bowl, take bowl, go trashcan", "Bowl_89852f2b"), 
                ("find bread, take bread, go trashcan", "Bread_460cf5d5"), 
                ("find vase, take vase, go trashcan", "Vase_3f629a7f")]

## Set Configs

In [7]:
is_cuda = torch.cuda.is_available()

if is_cuda: device = torch.device('cuda')
else: device = torch.device('cpu')

In [8]:
BUFFER_SIZE = int(1e4) # 2e4
BATCH_SIZE = 64
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
NUM_EPISODES = 12000 # 6000
TARGET_UPDATE = 4 # 8

TAU = 1e-4 # 5e-5
LR = 1.0e-4 # 1.0e-4

SCREEN_WIDTH = SCREEN_HEIGHT = 64

In [9]:
action_space = ["MoveAhead", "RotateLeft", "RotateRight", "PickupObject"]

## Load Word2Vec

In [10]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [11]:
glove_input_file = 'weights/glove.6B.100d.txt'
word2vec_output_file = 'weights/glove.6B.100d.txt.word2vec'

glove2word2vec(glove_input_file, word2vec_output_file)
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [12]:
pretrained_embeddings = torch.FloatTensor(word2vec_model.vectors)

## Set Replay Memory

In [13]:
class ReplayBuffer(object):
    """Fixed-size buffer to store experience tuples."""
    
    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayMemory object."""
        
        self.experience = namedtuple("Experience", field_names=["visual_state", "text_state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
        
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
    
    def add(self, visual_state, text_state, action, reward, next_state, done):
        """Add a new experience to buffer."""
        
        self.memory.append(self.experience(visual_state, text_state, action, reward, next_state, done))
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        
        experiences = random.sample(self.memory, k=self.batch_size)
        
        visual_states = torch.from_numpy(np.vstack([exp.visual_state.cpu().numpy() for exp in experiences if exp is not None])).float()
        visual_states = visual_states.to(device)
        
        text_states = torch.from_numpy(np.vstack([exp.text_state.cpu().numpy() for exp in experiences if exp is not None])).float()
        text_states = text_states.to(device)
        
        actions = torch.from_numpy(np.vstack([exp.action.cpu().numpy() for exp in experiences if exp is not None])).long()
        actions = actions.to(device)
        
        rewards = torch.from_numpy(np.vstack([exp.reward.cpu().numpy() for exp in experiences if exp is not None])).float()
        rewards = rewards.to(device)
        
        next_states = torch.from_numpy(np.vstack([exp.next_state.cpu().numpy() for exp in experiences if exp is not None])).float()
        next_states = next_states.to(device)
        
        dones = torch.from_numpy(np.vstack([exp.done for exp in experiences if exp is not None]).astype(np.uint8)).float()
        dones = dones.to(device)
        
        return (visual_states, text_states, actions, rewards, next_states, dones)
    
    def __len__(self):
        """Return the current size of internal memory."""
        
        return len(self.memory)

## Build Model

In [14]:
class VisualModel(nn.Module):
    
    def __init__(self, seed):
        
        super(VisualModel, self).__init__()
        
        self.seed = torch.manual_seed(seed)
        
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()
        
    def forward(self, x):
        
        return self.cnn(x)
    
    
class TextModel(nn.Module):
    
    def __init__(self, pretrained_embedding, hidden_dim, seed):
        
        super(TextModel, self).__init__()
        
        self.seed = torch.manual_seed(seed)
        
        self.embedding = nn.Embedding.from_pretrained(pretrained_embedding, freeze=True)
        self.rnn = nn.LSTM(pretrained_embedding.shape[1], hidden_dim)
        
    
    def forward(self, x):
        
        x = x.long()
        
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = output.view(output.shape[0], -1)
        
        return output

In [15]:
class MultimodalDQN(nn.Module):
    
    def __init__(self, visual_model, text_model, action_size, seed):
        
        super(MultimodalDQN, self).__init__()
        
        self.seed = torch.manual_seed(seed)
        
        self.visual_model = visual_model
        self.text_model = text_model
        
        # Define three fully connected layers
        self.fc1 = nn.Linear(640, 512) # 256, 512, 1024
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, action_size)
        
    def forward(self, visual_input, text_input):
        
        visual_features = self.visual_model(visual_input.to(device))
        text_features = self.text_model(text_input.to(device))
        
        # Concatenate visual and text features
        combined_features = torch.cat((visual_features, text_features), dim=1)
        
        # Apply fully connected layers
        combined_features = F.relu(self.fc1(combined_features))
        combined_features = F.relu(self.fc2(combined_features))
        q_values = self.fc3(combined_features)
        
        return q_values

## Build Agent

In [16]:
class DQNAgent():
    """The agent interacting with and learning from the environment."""
    
    def __init__(self, screen_width, screen_height, action_size, seed):
        """Init Agent’s models."""
        
        self.action_size = action_size
        self.seed = random.seed(seed)
        
        # Multimodal DQN
        self.visual_model = VisualModel(seed=seed)
        self.text_model = TextModel(pretrained_embeddings, hidden_dim=64, seed=seed)

        self.dqn_net = MultimodalDQN(self.visual_model, self.text_model, action_size, seed).to(device)
        self.target_net = MultimodalDQN(self.visual_model, self.text_model, action_size, seed).to(device)
        self.optimizer = optim.RMSprop(self.dqn_net.parameters(), lr=LR, alpha=0.95, eps=0.01)
        
        # Replay Buffer
        self.buffer = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        self.time_step = 0
    
    def memorize(self, visual_state, text_state, action, reward, next_state, done):
        """Save experience in replay buffer."""
        
        self.buffer.add(visual_state, text_state, action, reward, next_state, done)
    
        self.time_step = (self.time_step + 1) % TARGET_UPDATE
        if self.time_step == 0:
            # if enough samples are available in memory, get random subset and learn
            if len(self.buffer) > BATCH_SIZE:
                experiences = self.buffer.sample()
                self.learn(experiences, GAMMA)
    
    def visual_preprocess(self, visual_state, screen_width, screen_height):
        """Preprocess input frame before passing into agent."""
        
        resized_screen = Image.fromarray(visual_state).resize((screen_width, screen_height))
        visual_state = frame2tensor(to_torchdim(resized_screen)).to(torch.float32).to(device)

        return visual_state
    
    def text_preprocess(self, instruction):
        """Preprocess instructions before passing into agent."""
        
        text_state = instruction
        text_state = [word2vec_model.key_to_index[word] for word in text_state.split()]
        text_state = torch.tensor(text_state).long()
        text_state = text_state.unsqueeze(0)
        
        return text_state
    
    def act(self, visual_state, text_state, epsilon=0.):
        """Returns actions for given state as per current policy."""
        
        # epsilon-greedy action selection
        if random.random() > epsilon:
            self.dqn_net.eval()
            with torch.no_grad():
                action = self.dqn_net(visual_state, text_state).max(1)[1].view(1, 1)
            self.dqn_net.train()
            
        else:
            action = torch.tensor([[random.randrange(self.action_size)]], dtype=torch.long, device=device)
            
        return action
    
    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples."""
    
        visual_states, text_states, actions, rewards, next_states, dones = experiences
        
        self.optimizer.zero_grad()
        
        # get index of maximum value for next state
        Qsa_next = self.dqn_net(next_states, text_states).detach()
        _, action_max = Qsa_next.max(1)
        
        # get max predicted Q values (for next states) from target network
        Q_target_next = self.target_net(next_states, text_states).detach().gather(1, action_max.unsqueeze(1))
        
        # compute Q target
        Q_target = rewards + (gamma * Q_target_next * (1 - dones))
        
        # get expected Q values from dqn network
        Q_expected = self.dqn_net(visual_states, text_states).gather(1, actions)
        
        # compute loss
        loss = F.mse_loss(Q_target, Q_expected)
        
        # minimize the loss
        loss.backward()
        self.optimizer.step()
        
        # update target network
        self.soft_update(self.target_net, self.dqn_net, TAU)
    
    def soft_update(self, target_net, dqn_net, tau):
        """Soft update target network parameters."""
        
        for target_param, dqn_param in zip(target_net.parameters(), dqn_net.parameters()):
            target_param.data.copy_(tau*dqn_param.data + (1.0-tau) * target_param.data)
    
    def randomize_agent(self, controller):

        positions = controller.step(
            action="GetReachablePositions"
        ).metadata["actionReturn"]

        position = random.choice(positions)
        controller.step(
            action="Teleport",
            position=position,
            rotation=dict(x=0, y=270, z=0),
            horizon=0,
            standing=True
        )
        
    def watch(self, controller, instruction, action_space, num_episodes=10):
        """Watch trained agent."""
        
        best_score = -np.inf
        
        data = [("switch", "LightSwitch_887b121a"), ("tomato", "Tomato_e65a6e2e"), ("garbage", "GarbageCan_d6916cf5")]
        object_dict = dict(data)

        for i_episode in range(1, num_episodes+1):

            # initialize the environment and state
            controller.reset(random=True)
            
            self.randomize_agent(controller)

            visual_state = agent.visual_preprocess(controller.last_event.frame, 
                                               screen_width=SCREEN_WIDTH, screen_height=SCREEN_HEIGHT)
        
            instruction, AGENT_TARGET = instruction, object_dict[instruction]
            text_state = agent.text_preprocess(instruction)
            
            total_score = 0

            self.dqn_net.eval()
            
            for time_step in range(1, 50):
                
                clear_output(wait=True)
                
                # select an action using the trained dqn network
                with torch.no_grad():
                    action = self.dqn_net(visual_state, text_state).max(1)[1].view(1, 1)

                print(f"Time Step: {time_step}, Action: {action_space[action.item()]}")
                event = controller.step(action = action_space[action.item()])

                time.sleep(1.25)
                
                _, reward, done, _ = encode_feedback(event, controller, target_name=AGENT_TARGET)

                # observe a new state
                if not done:
                    screen = controller.last_event.frame
                    resized_screen = Image.fromarray(screen).resize((SCREEN_WIDTH, SCREEN_HEIGHT))

                    next_state = frame2tensor(to_torchdim(resized_screen)).to(torch.float32).to(device)
                else:
                    time.sleep(1.25)
                    next_state = None

                visual_state = next_state
                total_score += reward
                if done:
                    break

            if total_score > best_score: 
                best_score = total_score

            print(f'\rEpisode {i_episode}/{num_episodes}, Total Score: {total_score}, Best Score: {best_score}', end='') 

In [17]:
agent = DQNAgent(screen_width=SCREEN_WIDTH, screen_height=SCREEN_HEIGHT, action_size=len(action_space), seed=90)

In [18]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return {"Total": total_params, "Trainable": trainable_params}

param_counts = count_parameters(agent.dqn_net)
print(f"Total Parameters: {param_counts['Total']}")
print(f"Trainable Parameters: {param_counts['Trainable']}")

Total Parameters: 51679556
Trainable Parameters: 11679556


## Modify Taks

In [19]:
instructions = [("find bowl, take bowl, go trashcan, throw bowl", "Bowl_89852f2b"), 
                ("find bread, take bread, go trashcan, throw bread", "Bread_460cf5d5"), 
                ("find vase, take vase, go trashcan, throw vase", "Vase_3f629a7f")]

In [20]:
action_space = ["MoveAhead", "RotateLeft", "RotateRight", "PickupObject", "ThrowObject"]
action_size = len(action_space)

In [21]:
agent.dqn_net.load_state_dict(torch.load(f'./agents/AI2THOR_MM_RL_3OBJ_TASK3GEN.pth'));
agent.target_net.load_state_dict(torch.load(f'./agents/AI2THOR_MM_RL_3OBJ_TASK3GEN.pth'));

In [22]:
agent.action_size = action_size

agent.dqn_net.fc3 = nn.Linear(256, action_size); agent.dqn_net.to(device);
agent.target_net.fc3 = nn.Linear(256, action_size); agent.target_net.to(device);

## Train Agent

In [23]:
# define linear decay
def calculate_epsilon(episode):
    slope = (EPS_END - EPS_START) / NUM_EPISODES
    epsilon = EPS_START + slope * episode

    return max(epsilon, EPS_END)

In [24]:
def randomize_agent(controller):
    
    positions = controller.step(
        action="GetReachablePositions"
    ).metadata["actionReturn"]

    position = random.choice(positions)
    event = controller.step(
        action="Teleport",
        position=position,
        rotation=dict(x=0, y=270, z=0),
        horizon=0,
        standing=True
    )
    
    return event

In [25]:
def get_object_id_by_name(items, name):
    for item in items:
        if item['name'].startswith(name):
            return item['objectId']
    return None

In [26]:
def train_network(num_episodes, max_time):
    
    epsilon = EPS_START
    
    for i_episode in range(1, num_episodes+1):
        
        # initialize the environment and state
        controller.reset(random=True)
        
        event = randomize_agent(controller)
        
        visual_state = agent.visual_preprocess(controller.last_event.frame, 
                                               screen_width=SCREEN_WIDTH, screen_height=SCREEN_HEIGHT)
        
        inst_tupple = instructions[i_episode % len(instructions)]
        instruction, AGENT_TARGET = inst_tupple
        
        state1, state2, state3, state4 = instruction.split(',')
        
        text_state1 = agent.text_preprocess(state1.strip()) # find bowl
        text_state2 = agent.text_preprocess(state2.strip()) # take bowl
        text_state3 = agent.text_preprocess(state3.strip()) # go trashcan
        text_state4 = agent.text_preprocess(state4.strip()) # throw it

        text_state = text_state1
        
        is_first_visit = False
        is_first_pick = False
        is_first_go = False
        
        total_score = 0
        
        for time_step in range(1, max_time+1):
            
            # select and perform an action using dqn network
            action = agent.act(visual_state, text_state, epsilon)
            scalar_action = action.item()
            
            
            if (scalar_action == 4):
                
                AGENT_PLACE = "GarbageCan_d6916cf5"
                    
                event = controller.step(action=action_space[scalar_action], moveMagnitude=25.0, forceAction=False)
                _, reward, done, _ = encode_feedback_throw(event, controller, target_name=AGENT_TARGET, place_name=AGENT_PLACE)

            elif (scalar_action == 3):
                
                object_id = get_object_id_by_name(event.metadata['objects'], AGENT_TARGET)
                
                event = controller.step(action = action_space[scalar_action], 
                                        objectId = object_id, 
                                        forceAction=False, manualInteract=False)
                _, reward, done, _, is_first_pick = encode_feedback_pickup_for_go(event, controller, target_name=AGENT_TARGET, is_first=is_first_pick)
                
                target_data = next((obj for obj in event.metadata['objects'] if obj['name'] == AGENT_TARGET), None)
                
                if event.metadata['lastActionSuccess'] == True:
                    if target_data['isPickedUp'] == True:
                        text_state = text_state3
                    else: 
                        text_state = text_state2
            else:
                if text_state is text_state1 or text_state is text_state2:
                    event = controller.step(action = action_space[scalar_action])
                    _, reward, done, _, is_first_visit = encode_feedback_find_for_pickup(event, controller, target_name=AGENT_TARGET, is_first=is_first_visit)

                    data = controller.last_event.metadata['objects']

                    index_location = next((index for index, item in enumerate(data) if item['name'] == AGENT_TARGET), None)
                    is_found = data[index_location]['visible']

                    if is_found: text_state = text_state2
                    else: text_state = text_state1
                        
                elif text_state is text_state3 or text_state is text_state4:
                    
                    AGENT_PLACE = "GarbageCan_d6916cf5"
                    
                    event = controller.step(action = action_space[scalar_action])
                    _, reward, done, _, is_first_go = encode_feedback_go_for_throw(event, controller, place_name=AGENT_PLACE, is_first=is_first_go)
                    
                    data = controller.last_event.metadata['objects']

                    index_location = next((index for index, item in enumerate(data) if item['name'] == AGENT_PLACE), None)
                    is_found = data[index_location]['visible']

                    if is_found: text_state = text_state4
                    else: text_state = text_state3

            # time.sleep(1)
            
            total_score += reward
            reward = torch.tensor([reward], device=device)
            
            next_state = agent.visual_preprocess(controller.last_event.frame, 
                                                 screen_width=SCREEN_WIDTH, screen_height=SCREEN_HEIGHT)
            
            agent.memorize(visual_state, text_state, action, reward, next_state, done)
            
            # move to the next state
            visual_state = next_state
             
            if done or time_step == max_time:
                episode_scores = plot_durations_w_shaded(total_score, i_episode, num_episodes, instruction, total_score, time_step)
                break
            
        epsilon = calculate_epsilon(i_episode)
        
    if not os.path.exists('./agents/'): os.makedirs('./agents/')
    torch.save(agent.dqn_net.state_dict(), f'./agents/AI2THOR_MM_RL_3OBJ_TASK4GENc.pth')
    
    # print('Training completed.')
    plt.ioff()
    plt.show()
    
    return episode_scores

In [27]:
print('Training the network...')
episode_scores = train_network(num_episodes=NUM_EPISODES, max_time=200)

## Analyze Results

In [28]:
first_es = episode_scores[0::3] # bowl
second_es = episode_scores[1::3] # bread
third_es = episode_scores[2::3] # vase

plt.figure(figsize=(8, 6))
plt.clf()
scores_t = torch.tensor(first_es, dtype=torch.float)

plt.title(f'Training Results - Bowl, Bread, Vase')
plt.xlabel('Episode')
plt.ylabel('Cumulative Rewards')

# take 20 episode averages and plot them
window_size = 20

# calculate fixed-size window averages (means) and standard deviations
means = scores_t.unfold(0, window_size, 1).mean(1).view(-1)
std_deviation = scores_t.unfold(0, window_size, 1).std(1).view(-1)

# pad the start with NaNs to align the plots correctly
means = torch.cat((torch.zeros(window_size-1), means))
std_deviation = torch.cat((torch.zeros(window_size-1), std_deviation))

# calculate the x-axis indices for plotting
x_values = torch.arange(len(scores_t))

# plot means with standard deviation as shaded area
plt.plot(x_values.numpy(), means.numpy(), label='Average Reward (Bowl)')
plt.fill_between(x_values.numpy(), (means-std_deviation).numpy(), (means+std_deviation).numpy(), alpha=0.2, label='Standard Deviation (Bowl)')

scores_t = torch.tensor(second_es, dtype=torch.float)

# calculate fixed-size window averages (means) and standard deviations
means = scores_t.unfold(0, window_size, 1).mean(1).view(-1)
std_deviation = scores_t.unfold(0, window_size, 1).std(1).view(-1)

# pad the start with NaNs to align the plots correctly
means = torch.cat((torch.zeros(window_size-1), means))
std_deviation = torch.cat((torch.zeros(window_size-1), std_deviation))

# calculate the x-axis indices for plotting
x_values = torch.arange(len(scores_t))

# plot means with standard deviation as shaded area
plt.plot(x_values.numpy(), means.numpy(), label='Average Reward (Bread)')
plt.fill_between(x_values.numpy(), (means-std_deviation).numpy(), (means+std_deviation).numpy(), alpha=0.2, label='Standard Deviation (Bread)')

scores_t = torch.tensor(third_es, dtype=torch.float)

# calculate fixed-size window averages (means) and standard deviations
means = scores_t.unfold(0, window_size, 1).mean(1).view(-1)
std_deviation = scores_t.unfold(0, window_size, 1).std(1).view(-1)

# pad the start with NaNs to align the plots correctly
means = torch.cat((torch.zeros(window_size-1), means))
std_deviation = torch.cat((torch.zeros(window_size-1), std_deviation))

# calculate the x-axis indices for plotting
x_values = torch.arange(len(scores_t))

# plot means with standard deviation as shaded area
plt.plot(x_values.numpy(), means.numpy(), label='Average Reward (Vase)')
plt.fill_between(x_values.numpy(), (means-std_deviation).numpy(), (means+std_deviation).numpy(), alpha=0.2, label='Standard Deviation (Vase)')

# place the legend
plt.legend()
plt.savefig('./images/plot_of_training_result_AI2THOR_RL_ATT_3OBJ_TASK4GENc.png')
    
plt.show()

## Save Data

In [29]:
import pickle

In [30]:
lcurve_path = "./agents/lcurve/episode_scores_WO_ATT_3OBJ_TASK4GENc"

with open(lcurve_path, "wb") as fp:
    pickle.dump(episode_scores, fp)

with open(lcurve_path, "rb") as fp:
    episode_scores = pickle.load(fp)

---